<a href="https://colab.research.google.com/github/RenanCostaNascimento/mestrado-reconhecimento-padroes/blob/main/Avaliando_a_generaliza%C3%A7%C3%A3o_de_algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O objetivo deste Notebook é comparar classificadores. Os classificadores escolhidos são o [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) e o [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) do scikit-learn. Vamos começar importando a base de dados [Heart Disease UCI do Kaggle](https://www.kaggle.com/ronitf/heart-disease-uci). É um dataset que indica se a pessoa tem alguma condição cardíaca com base nas suas informações médicas.


In [138]:
import pandas as pd

dataset = pd.read_csv('/content/heart.csv')
dataset.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1


Os dados da base de dados devem ser pré-processados. Para tanto, vamos começar separando os dados em uma matriz de features (dados de entrada) e um vetor coluna com as classes (última coluna – “target” – do arquivo csv).

In [139]:
import numpy as np

# transforma o dataset do pandas em np array
featureMatrix = dataset.to_numpy()

# extrai a ultima coluna da matrix
labels = featureMatrix[:,len(featureMatrix[0]) - 1]

# remove a ultima coluna da featureMatrix
featureMatrix = np.delete(featureMatrix, len(featureMatrix[0]) - 1, 1)

np.shape(featureMatrix), np.shape(labels)

((303, 13), (303,))

Agora precisamos dividir nossa base de dados em treino e teste. Para fazer isso vamos utilizar a função [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) do sklearn. Em seguida precisamos de fato treinar nossos modelos. Para garantir que a comparação dos classificadores seja feita da melhor forma possível, vamos treinar o modelo de 10 formas diferentes, cada uma com um random_state único. Isso nos dará 20 modelos distintos, um para cada classificador. Para facilitar o trabalho, abaixo definimos uma função que separa a base de dados em teste e treino, além de treinar os modelos.

In [140]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

def trainTestSplit(randomState):
  return train_test_split(featureMatrix, labels, test_size=0.33, random_state=randomState)

def trainModels(featuresTrain, labelsTrain):
  # eu precisei colocar o max_iter=1000 porque estava dando esse erro sem ele:
  #  ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
  logisticModel = LogisticRegression(max_iter=10000)
  logisticModel.fit(featuresTrain, labelsTrain)

  knnModel = KNeighborsClassifier()
  knnModel.fit(featuresTrain, labelsTrain)

  return logisticModel, knnModel

Com a função de treinamento definida, vamos agora criar outra função para avaliar os resultados. A avaliação será feita utilizando as métricas [Mean Squared Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) e [F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html), ambas do sklearn.

In [141]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

def evaluateModel(model, featuresTest, labelsTest):
  prediction = model.predict(featuresTest)
  mse = mean_squared_error(labelsTest, prediction)
  f1Score = f1_score(labelsTest, prediction)

  return mse, f1Score

Por fim, vamos executar todo esse procedimento 10 vezes e tirar a média das métricas.

In [143]:
logisticErrors = []
logisticScores = []
knnErrors = []
knnScores = []

for test in range(10):
  featuresTrain, featuresTest, labelsTrain, labelsTest = trainTestSplit(test)
  logisticModel, knnModel = trainModels(featuresTrain, labelsTrain)

  logisticMse, logisticF1Score = evaluateModel(logisticModel, featuresTest, labelsTest)
  logisticErrors.append(logisticMse)
  logisticScores.append(logisticF1Score)

  knnMse, knnF1Score = evaluateModel(knnModel, featuresTest, labelsTest)
  knnErrors.append(knnMse)
  knnScores.append(knnF1Score)

print(f"Logistic Mean MSE: {np.mean(logisticErrors)}")
print(f"KNN Mean MSE: {np.mean(knnErrors)}")
print("---------")
print(f"Logistic F1 Score: {np.mean(logisticScores)}")
print(f"KNN F1 Score: {np.mean(knnScores)}")

Logistic Mean MSE: 0.16799999999999998
KNN Mean MSE: 0.356
---------
Logistic F1 Score: 0.8537218191901557
KNN F1 Score: 0.6849915391281238
